In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "f_transaction_line")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_DWH_PRD.F_SALE_PROD, VCM_DWH_PRD.F_COGS"
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line (
  hash_id BIGINT,
  calday DATE,
  product_id STRING,
  base_uom STRING,
  sales_unit STRING,
  business_unit STRING,
  transaction_id STRING,
  transaction_no STRING,
  transaction_type STRING,
  vinid_card_no STRING,
  store_id STRING,
  return_qty DOUBLE,
  return_value DOUBLE,
  sale_qty DOUBLE,
  promotion_qty DOUBLE,
  promotion_sale DOUBLE,
  sale_amount DOUBLE,
  base_sale_qty DOUBLE,
  receipt_count DOUBLE,
  sale_in_cost_value DOUBLE,
  tax_amount DOUBLE,
  cogs_per_unit DOUBLE,
  cogs DOUBLE,
  sold_quantity DOUBLE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")


In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_f_transaction_line AS
    WITH cogs AS (
        SELECT  
            calday,
            store_id,
            product_id,
            cogs_per_unit,
            cost,
            quantity
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs
    )
    SELECT
        a.hash_id,
        a.calday,
        a.product_id,
        a.base_uom,
        a.sales_unit,
        a.business_unit,
        a.transaction_id,
        a.transaction_no,
        a.transaction_type,
        a.vinid_card_no,
        a.store_id,
        a.return_qty2 AS return_qty,
        a.return_value,
        a.sale_qty,
        a.promotion_qty,
        a.promotion_sale,
        a.sale_amount,
        a.base_sale_qty,
        a.receipt_count,
        a.sale_in_cost_value,
        a.tax_amount,
        a1.cogs_per_unit,
        a1.cogs_per_unit * a.base_sale_qty AS cogs,
        a1.quantity AS sold_quantity
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
    LEFT JOIN cogs a1 USING (calday, store_id, product_id)
    WHERE product_id IS NOT NULL
    AND base_sale_qty != 0 
    AND calday in ({CALDAY_IN_STR})
    """)
   
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != '':
    # Get the calday list string from temp table
    calday_query = """
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_f_transaction_line
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line
        WHERE calday IN ({calday_in_str})
        """)
    else:
        print("No valid CALDAYs found to delete.") 

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
spark.sql(f"DROP VIEW IF EXISTS f_transaction_line")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"